In [1]:
from wmfdata import hive, mariadb
from wmfdata.utils import num_str

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


# Hive

In [ ]:
hive.run?

In [ ]:
cawiki_active_eds = hive.run("""
select
    event_user_text
from wmf.mediawiki_history
where
    wiki_db = "cawiki" and
    event_timestamp between "2018-10" and "2018-11" and
    not event_user_is_anonymous and
    event_entity = "revision" and
    event_type = "create" and
    snapshot = "2018-10"
group by event_user_text
""")

In [ ]:
cawiki_active_eds.tail()

In [ ]:
cawiki_active_eds["event_user_text"].str.len().mean()

# MariaDB

In [10]:
wikipedias = mariadb.list_wikis("wikipedia")
wikipedias[:5]

['aawiki', 'abwiki', 'acewiki', 'adywiki', 'afwiki']

In [14]:
wp_articles = mariadb.multirun("""
select
    database() as wiki,
    ss_good_articles as articles
from site_stats

""", wikis=wiktionaries)

aawiki completed in 0 s
abwiki completed in 0 s
acewiki completed in 0 s
adywiki completed in 0 s
afwiki completed in 0 s
akwiki completed in 0 s
alswiki completed in 0 s
amwiki completed in 0 s
angwiki completed in 0 s
anwiki completed in 0 s
arcwiki completed in 0 s
arwiki completed in 0 s
arzwiki completed in 0 s
astwiki completed in 0 s
aswiki completed in 0 s
atjwiki completed in 0 s
avwiki completed in 0 s
aywiki completed in 0 s
azbwiki completed in 0 s
azwiki completed in 0 s
barwiki completed in 0 s
bat_smgwiki completed in 0 s
bawiki completed in 0 s
bclwiki completed in 0 s
be_x_oldwiki completed in 0 s
bewiki completed in 0 s
bgwiki completed in 0 s
bhwiki completed in 0 s
biwiki completed in 0 s
bjnwiki completed in 0 s
bmwiki completed in 0 s
bnwiki completed in 0 s
bowiki completed in 0 s
bpywiki completed in 0 s
brwiki completed in 0 s
bswiki completed in 0 s
bugwiki completed in 0 s
bxrwiki completed in 0 s
cawiki completed in 0 s
cbk_zamwiki completed in 0 s
cdowiki c

In [25]:
wp_articles.head(10)

,wiki,articles
0,aawiki,1
1,abwiki,3428
2,acewiki,7456
3,adywiki,411
4,afwiki,67002
5,akwiki,566
6,alswiki,25234
7,amwiki,14433
8,angwiki,3054
9,anwiki,33884


In [16]:
n_articles = wp_articles["articles"].sum()
n_articles

49154858

In [19]:
print(num_str(n_articles, n_figs=3))

49,200,000


In [21]:
filename = "/home/neilpquinn-wmf/wp_articles.csv"
wp_articles.to_csv(filename, index=False)

In [22]:
hive.load_csv(filename, field_spec="wiki string, articles int", db_name="neilpquinn", table_name="article_counts")

log4j:WARN No such property [maxBackupIndex] in org.apache.log4j.DailyRollingFileAppender.

Logging initialized using configuration in file:/etc/hive/conf.analytics-hadoop/hive-log4j.properties
Loading data to table neilpquinn.article_counts
OK
Time taken: 1.865 seconds



In [24]:
hive.run("select * from neilpquinn.article_counts limit 10")

,wiki,articles
0,aawiki,1
1,abwiki,3428
2,acewiki,7456
3,adywiki,411
4,afwiki,67002
5,akwiki,566
6,alswiki,25234
7,amwiki,14433
8,angwiki,3054
9,anwiki,33884
